# Task

Create videos based on images in the output directories

TODO: make origin and data folders a function entry


In [1]:
def main():

    # importing libraries
    import os
    import cv2 
    from PIL import Image 
    import csv
    import glob
    import itertools

    data    = ['processed']         #  the state of the data
    sensor  = ['LWIR','Visible']    #  names of the sensors in the scenario
    ftype   = ['txt']               #  exported data format

    # data    = ['raw']               #  the state of the data
    # sensor  = ['LWIR','Visible']    #  names of the sensors in the scenario
    # ftype   = ['bitmap']            #  exported data format

    # cld     = ['Off','On','On','On']
    # atm     = [0,0,1,2]

    cld     = ['Off','On','On']
    atm     = [0,0,2]

    origin = "C:/Users/nkazmi/Documents/STK 12/EOIR_Clouds_AirToAir - Python - new flight/testing ground/"
    name = []
    for (a,b,c) in itertools.product(data,ftype,sensor):
        for (x,y) in itertools.zip_longest(cld,atm):
            name.append("{}{}_{}_{}_Cld{}_Atm{}".format(origin, a, b, c, x, y))
            
    for i in range(len(name)):
        #print(name[i])
        # Checking the current directory path
        #print(os.getcwd()) 
        
        # Folder which contains all the images
        # from which video is to be generated
        os.chdir(name[i])
        path = name[i]
        
        mean_height = 0
        mean_width = 0
        
        num_of_images = len(os.listdir('.'))
        # print(num_of_images)

        for file in os.listdir('.'):
            im = Image.open(os.path.join(path, file))
            width, height = im.size
            mean_width += width
            mean_height += height
            #im.show()   # uncomment this for displaying the image

        # Finding the mean height and width of all images.
        # This is required because the video frame needs
        # to be set with same width and height. Otherwise
        # images not equal to that width height will not get 
        # embedded into the video
        mean_width = int(mean_width / num_of_images)
        mean_height = int(mean_height / num_of_images)
        
        #print(mean_height)
        #print(mean_width)

        # Resizing of the images to give
        # them same width and height 
        for file in os.listdir('.'):
            if file.endswith(".jpg") or file.endswith(".jpeg") or file.endswith(".bmp"):
                # opening image using PIL Image
                im = Image.open(os.path.join(path, file)) 
        
                # im.size includes the height and width of image
                width, height = im.size   
                #print(width, height)
        
                # resizing 
                imResize = im.resize((mean_width, mean_height), Image.ANTIALIAS) 
                imResize.save( file, 'JPEG', quality = 99) # setting quality
                # printing each resized image name
                #print(im.filename.split('\\')[-1], " is resized") 
        
        # removing extra parts from the video name
        discard = len(origin) + len(data[0]) + len(ftype[0]) + 2
        
        # Video Generating function
        def generate_video():
            image_folder = '.' # make sure to use your folder
            #print(name[i][discard:])
            video_name = name[i][discard:] + ".mp4"
            os.chdir(name[i])
            
            images = [img for img in os.listdir(image_folder)
                    if img.endswith(".jpg") or
                        img.endswith(".jpeg") or
                        img.endswith(".bmp")]
            
            # Array images should only consider
            # the image files ignoring others if any
            # sort images by numerical value

            # Only sort for .bmp
            if ftype == ['bitmap']:
                images = sorted([images for images in os.listdir('.') \
                    if images.endswith('.bmp')], \
                    key=lambda f: int(f.rsplit(os.path.extsep, 1)[0].rsplit(None,1)[-1]))
                # print(images)
            else:
                pass

            frame = cv2.imread(os.path.join(image_folder, images[0]))
        
            # setting the frame width, height width
            # the width, height of first image
            height, width, layers = frame.shape  
    
            # save one directory above the image folder 
            video = cv2.VideoWriter(str(origin + video_name), 0, 10, (width, height)) 
        
            # Appending the images to the video one by one
            for image in images: 
                video.write(cv2.imread(os.path.join(image_folder, image))) 
            
            # Deallocating memories taken for window creation
            cv2.destroyAllWindows()
            video.release()  # releasing the video generated
        
        # Calling the generate_video function
        generate_video()

if __name__ == '__main__':
    main()       